### Imports

In [1]:
from __future__ import annotations

from pathlib import Path

from kub.course.plotlib.simulationPlotFactory import SimulationPlotFactory
from kub.course.simlib.simulation import FMUSimulation

# Find repository root: check if database/ exists at current location
repo_root = (
    Path.cwd() if (Path.cwd() / "database").exists() else Path.cwd().parent.parent
)
databasePath = repo_root / "database" / "day1" / "FMUs"

### **Exercise 8.1: Loss Assesment**

<img src="../../database/images/LossAssesment8_1.png"
     align="right"
     width="600"
     style="margin-right: 20px; margin-bottom: 10px;" />

You are analyzing the required heating load for a building to ensure the internal air temperature never drop below 19°C.
A heating component is integrated into the system to force this temperature and supply the necessary heat flux to compensate for thermal losses.

You are provided with a graph showing the net heat flux required by the heating component over a 7 days period.

**Heating Load (Boiler Sizing)**: In the next part of the exercise we will provide you a model with a boiler,
- Find the heating power necessary to avoid dropping below the **minimum temperature of 19°C**.

**Physical Interpretation of Negative Flux:**
- **Explain the physical reason** why the net heat flux  becomes **negative** during certain hours (i.e., when the temperature control component must remove heat).

In [3]:
sim1 = FMUSimulation(
    databasePath / "Exercises_LossAssesment_SolarGainFluxInjectionSizing_8_1.fmu"
)

sim1.initialize(startTime=0.0, stopTime=86400 * 7, timeStep=3600.0)
sim1.initParameters(
    {
        "temperatureSetPoint": 19.0,
    }
)  # modify temperature set point
sim1.exitInitialization()

outputNames = [
    "prescribedTemperature.port.T",
    "prescribedTemperature1.port.T",
    "powerOutputBoiler",
    "solarFluxGenerator.solarRadiationOut.IrrDir",
]
data1 = sim1.run(outputNames)

sim1Factory = SimulationPlotFactory()

sim1Factory.plot_multi_curves(
    time=data1["time"],
    data_type="heat_flux",
    data_dict={"powerOutputBoiler": data1["powerOutputBoiler"]},
    title="Exercise 8.1 — Power Needed to Maintain temperatureSetPoint",
)

sim1Factory.plot_multi_curves(
    time=data1["time"],
    data_type="temperature",
    data_dict={
        "Exterior Temperature": data1["prescribedTemperature.port.T"],
        "Interior Temperature": data1["prescribedTemperature1.port.T"],
    },
    title="Exercise 8.1 — Evolution of Temperature",
)

sim1Factory.plot_multi_curves(
    time=data1["time"],
    data_type="solar_radiation",
    data_dict={"Solar Radiation": data1["solarFluxGenerator.solarRadiationOut.IrrDir"]},
    title="Exercise 8.1 — Solar Radiation",
);

### **Exercise 8.2: Loss Assesment**

<img src="../../database/images/LossAssesment8_2.png"
     align="right"
     width="600"
     style="margin-right: 20px; margin-bottom: 10px;" />

Input the correct sizing for the boiler. The boiler is initially regulated by a constant block set to 1. This forces the boiler to always produce its maximum amount of heat.

**Initial Analysis (Maximum Heat Output):**
- Analyze the provided **air temperature curve**. Does the measured air temperature comply with the requirement of maintaining a minimum temperature of 19°C ?

**Problem Identification:**
- Assuming the 19°C requirement is met, what is the **main problem** evident from the temperature curve under this maximum heat output regulation?

**Regulation Strategy:**
- Describe how you can use the **constant block** (with an adjustable value between **0 and 1**) to properly **regulate the boiler's heat output** and address the problem identified in step 3.

In [8]:
sim2 = FMUSimulation(
    databasePath / "Exercises_LossAssesment_SolarGainFluxInjection_8_2.fmu"
)

sim2.initialize(startTime=0.0, stopTime=86400 * 7, timeStep=3600.0)
sim2.initParameters(
    {
        "boiler.heatingPower": 300.0,  # to modify
        "const.k": 1.0,
    }
)  # to modify
sim2.exitInitialization()

outputNames = ["prescribedTemperature.port.T", "AirTemperature"]
data2 = sim2.run(outputNames)

sim2Factory = SimulationPlotFactory()

sim2Factory.plot_multi_curves(
    time=data2["time"],
    data_type="temperature",
    data_dict={
        "Exterior Temperature": data2["prescribedTemperature.port.T"],
        "AirTemperature": data2["AirTemperature"],
    },
    title="Exercise 8.2 — Evolution of Temperature",
);

### **Exercise 8.3: Loss Assesment**

<img src="../../database/images/LossAssesment8_3.png"
     align="right"
     width="600"
     style="margin-right: 20px; margin-bottom: 10px;" />

We regulate the model using a **PID controller**. Essentially, this controller receives two main inputs: a **setpoint** (the desired value) and a **measured signal** (the actual value). Based on the difference between these two, the controller calculates and provides an **output signal** that ranges from 0 to 1. This output is then used to adjust the system, with the goal of bringing the measured signal as close as possible to the setpoint.

**Initial Simulation and Analysis**
- **Input the correct sizing** for the boiler.
- The **temperature setpoint** for the PID controller is initially set to **19°C**.
- Run the simulation and **observe the air temperature curve**.
    - Does the air temperature perfectly maintain 19°C ?
    - **Explain why** the temperature may rise above the setpoint.
- **Observe the power output** of the boiler during this simulation. Compare this power output to the power output observed in the previous exercise (when the constant block was set to 1).

**Setpoint Change and Re-analysis**
- Change the temperature setpoint in the PID controller to 20°C.
- Run the simulation and observe the air temperature curve again.
    - Does the air temperature perfectly maintain the new 20°C setpoint?
- **Observe the power output** of the boiler.
    - What issue do you observe in the boiler's power output that prevents the system from reaching the desired 20°C?
**What action** must be taken to ensure the system can effectively maintain a 20°C setpoint against thermal losses?

Compare **energy consumption** between a **19°C setpoint** and a **20°C setpoint** what **% gain** have we made by reducing the temperature by 1°C.


In [18]:
sim3 = FMUSimulation(
    databasePath / "Exercises_LossAssesment_SolarGainFluxInjectionRegulated_8_3.fmu"
)

sim3.initialize(startTime=0.0, stopTime=86400 * 7, timeStep=3600.0)
sim3.initParameters(
    {
        "temperatureSetPoint": 20.0,  # to modify
        "boiler.heatingPower": 500.0,
    }
)  # to modify
sim3.exitInitialization()

outputNames = [
    "prescribedTemperature.port.T",
    "AirTemperature",
    "powerOutputBoiler",
    "energyBoiler_kWh",
]
data3 = sim3.run(outputNames)

print(f"Total Consumption: {data3["energyBoiler_kWh"][-1]}")

sim3Factory = SimulationPlotFactory()

sim3Factory.plot_multi_curves(
    time=data3["time"],
    data_type="temperature",
    data_dict={
        "Exterior Temperature": data3["prescribedTemperature.port.T"],
        "Interior Temperature": data3["AirTemperature"],
    },
    title="Exercise 8.3 — Evolution of Temperature",
)

sim3Factory.plot_multi_curves(
    time=data1["time"],
    data_type="heat_flux",
    data_dict={"powerOutputBoiler": data3["powerOutputBoiler"]},
    title="Exercise 8.3 — Power Output of Boiler",
)

sim3Factory.plot_multi_curves(
    time=data1["time"],
    data_type="energy",
    data_dict={"energyBoiler_kWh": data3["energyBoiler_kWh"]},
    title="Exercise 8.3 — Evolution of Energy Consumption",
);


Total Consumption: 32.12448701086712
